# d) Setup Environmental Flow Requirements

Calculated Environmental Flow exceedance percentiles for each day of the historic flow series. These are used later to determine environmental flows.

## Procedure

From: **Mott MacDonald** - *Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D*

Based on the environmental flow bands, the methodology adopted for the energy modelling is as below:

- The flow on the 8th, 18th and 28th day of each month is identified. 
- The equivalent percentile of this flow based on flow exceedance curves is calculated for that day from the 92 years of data.
- This percentile is used to decide the environmental flow release for the succeeding ~10 day periods of 1st to 10th, 11th to 20th and 21st to end of the month.
- Environmental flow in this band and month is released in the respective channels.

In this notebook the exceedance percentiles for each day of the historic flow series are calculated. These will later be used to lookup the required EWR values for each channel depending on the Environmental Assurance Category selected for the channel. 

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| ngonye_flow_daily.csv        | Notebook: c_flow_data | Daily Flow series              |



## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| ngonye_flow_daily_ewrx.csv  | Daily flow series for Ngonye with EWR Exceedance values  |


## Parameters

In [1]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [2]:
import numpy as np
import pandas as pd

## Load Daily Data

In [3]:
daily=pd.read_csv(output_data + "ngonye_flow_daily.csv")
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)

## Build FDCs
Build FDCs for each of the 'measurement days' - 8th, 18th, 28th of each month

In [4]:
cols=['9-28','10-8','10-18','10-28','11-8','11-18','11-28','12-8','12-18','12-28',
      '1-8','1-18','1-28','2-8','2-18','2-28','3-8','3-18','3-28','4-8','4-18',
      '4-28','5-8','5-18','5-28','6-8','6-18','6-28','7-8','7-18','7-28','8-8','8-18','8-28','9-8','9-18']
ewr_fdcs=pd.DataFrame(columns=cols,index=np.arange(0,1.01,0.01))

for column in ewr_fdcs:   
    (month,day)=column.split('-')
    ewr_fdcs[column]=np.percentile(daily[(daily['Month']==int(month)) & (daily['Day']==int(day))]['Flow'] ,(1-ewr_fdcs.index)*100,interpolation='linear')
    
ewr_fdcs.tail(5)

,9-28,10-8,10-18,10-28,11-8,11-18,11-28,12-8,12-18,12-28,...,6-18,6-28,7-8,7-18,7-28,8-8,8-18,8-28,9-8,9-18
0.96,182.250972,166.252153,148.702978,155.208298,155.441976,175.842390,198.710389,240.309741,278.224952,341.692537,...,416.208238,368.958502,339.867541,322.534870,290.521945,268.291987,233.700848,219.873321,201.551427,192.851404
0.97,172.169726,148.886575,138.690683,147.120940,148.486385,172.936825,193.937364,230.789847,266.142637,321.180737,...,387.948680,348.393743,326.849258,306.748677,284.181923,258.233853,229.810440,212.795551,198.939428,187.275192
0.98,153.561443,126.407216,117.214572,124.555313,141.979000,162.918400,190.902616,221.325727,259.411218,274.460651,...,338.205784,307.333800,287.280393,262.917603,258.546771,236.583566,220.578489,204.013866,190.699301,178.389923
0.99,138.462710,123.439808,112.736223,116.337330,138.273815,145.015643,188.962733,194.673993,234.767880,265.427094,...,324.929726,289.385048,269.345078,255.127542,228.989562,219.630018,198.781771,189.604061,168.680377,154.926679
1.00,135.250360,111.407480,111.407480,106.765106,132.889173,141.159533,185.778706,193.274660,234.767880,249.799180,...,316.522424,276.310730,258.797652,237.242204,224.671047,209.274518,195.207704,179.826510,162.781303,145.350957


## Functions

In [5]:
#Gives the measure date (proceeding 8th, 18th or 28th) for the given date
def measureDate(obsDate):
    if obsDate.day <= 10:
        if obsDate.month==1:
            mn=13
            yr=obsDate.year-1
        else:
            mn=obsDate.month
            yr=obsDate.year
        return pd.Timestamp(yr, mn-1, 28)
    elif obsDate.day <=20:
        return pd.Timestamp(obsDate.year, obsDate.month, 8)
    else:
        return pd.Timestamp(obsDate.year, obsDate.month, 18)

#If it is a measure date (8th, 18th or 28th) then returns the ID of an FDC for that date. Otherwise False.
def isMeasureDay(obsDate):
    if obsDate.day==8 or obsDate.day==18 or obsDate.day==28:
        return str(obsDate.month) + '-' + str(obsDate.day) 
    else:
        return False
    
#Given an FDC ID and flow looks up exceedance
def lookupExceedance(measureDay,flow):
    return ewr_fdcs[ewr_fdcs[measureDay]<=flow].index.min()

## Exceedance Lookup
Build a table of measurement days and lookup the exceedance value for these from the FDCs already built

In [6]:
daily['dDate']=daily.index
daily['MeasureDay']=daily.apply(lambda x: isMeasureDay(x['dDate']),axis=1)
ewr_measure_days=pd.DataFrame(daily.loc[daily['MeasureDay']!=False]['MeasureDay'])
ewr_measure_days['Flow']=daily['Flow']
ewr_measure_days['Exceedance']=ewr_measure_days.apply(lambda x: lookupExceedance(x['MeasureDay'],x['Flow']),axis=1)
ewr_measure_days.tail(3)

,MeasureDay,Flow,Exceedance
Date,,,
2017-09-08,9-8,240.031205,0.80
2017-09-18,9-18,216.508407,0.82
2017-09-28,9-28,200.552607,0.83


## EWR Measure Dates
Mark the dates on the daily timeseries where the EWR will be re-measured (8th, 18th, 28th of each month) 

In [7]:
daily['dDate']=daily.index
daily['EWRMeasureDate']=daily.apply(lambda x: measureDate(x['dDate']),axis=1)

## Daily EWR Exceedance
Lookup the EWR exceedance values for each day of the daily series based on the measure date

In [8]:
daily['EWRRefExceedance']=daily.join(ewr_measure_days,on='EWRMeasureDate',lsuffix='_l')['Exceedance']

In [9]:
daily=daily.drop(['MeasureDay','EWRMeasureDate','dDate'],axis=1)
daily=daily.fillna(0.5)
daily.tail(3)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,WaterWeek,Volume,Flow_difference,EWRRefExceedance
Date,,,,,,,,,,,,,,,,
2017-09-28,2017-10-09,204.0855,0.982689,200.552607,0.955,2017,9,28,2017.09,2016,12,363,52,0.017328,0.000000,0.82
2017-09-29,2017-10-10,204.0855,0.982689,200.552607,0.955,2017,9,29,2017.09,2016,12,364,52,0.017328,0.000000,0.82
2017-09-30,2017-10-11,200.9197,0.995488,200.013130,0.958,2017,9,30,2017.09,2016,12,365,53,0.017281,0.539477,0.82


## Save

In [10]:
daily.to_csv(output_data + 'ngonye_flow_daily_ewrx.csv')